In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from sklearn.cluster import KMeans
from tqdm import tqdm
# import heapq
# from unsloth import FastLanguageModel
from peft import PeftModelForCausalLM

In [2]:
base_data = load_from_disk("/mnt/bn/data-tns-live-llm/leon/datasets/privacy_data/base_code_0.parquet/")
print(len(base_data))

100


In [ ]:
# model: PeftModelForCausalLM
# model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/base_code",dtype = torch.bfloat16,
#     load_in_4bit = True)
# FastLanguageModel.for_inference(model)

In [3]:
from vllm import LLM, SamplingParams
llm = LLM(model="/mnt/bn/data-tns-live-llm/leon/datasets/privacy/code_2000_merged/", tensor_parallel_size=1, 
              dtype=torch.bfloat16, trust_remote_code=True, 
              enable_lora=False, max_model_len=2048,gpu_memory_utilization=0.75)

INFO 08-16 22:57:01 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/code_2000_merged/', speculative_config=None, tokenizer='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/code_2000_merged/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/bn/data-tns-live-llm/leon/datasets/privacy/code_2000_merged/, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-16 22:57

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-16 22:57:05 model_runner.py:732] Loading model weights took 14.9595 GB
INFO 08-16 22:57:06 gpu_executor.py:102] # GPU blocks: 22106, # CPU blocks: 2048
INFO 08-16 22:57:07 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-16 22:57:07 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-16 22:57:24 model_runner.py:1225] Graph capturing finished in 16 secs.


In [45]:
# from transformers import TextStreamer
# streamer = TextStreamer(tokenizer)
# model.generation_config.pad_token_id = tokenizer.eos_token_id

In [6]:
import evaluate
rouge = evaluate.load('rouge')
stop_tokens = ["USER:", "ASSISTANT:",  "### Instruction:", "Response:", 
                "\n\nProblem", "\nProblem", "Problem:", "<|eot_id|>", "####"]
# sampling_params = SamplingParams(temperature=0.5, top_p=1, max_tokens=1024, stop=stop_tokens, repetition_penalty=1.1)
sampling_params = SamplingParams(temperature=0.5, top_p=1, max_tokens=1024, repetition_penalty=1.1, top_k=1) # when greedy decode, k=1

In [5]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [85]:
# def generate_one_completion(instance):
#     inputs = tokenizer(instance, return_tensors="pt")
#     input_ids = inputs.input_ids.to("cuda")
#     generate_ids = model.generate(input_ids, max_new_tokens=2048, repetition_penalty=1.1, streamer=streamer, do_sample=True)
#     outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     # print(outputs)
#     return outputs

# 遍历

In [ ]:
inputs = [template.format_map({"instruction":input["instruction"]}) for input in base_data]

In [4]:
prefix = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:"""
inputs = [prefix]*100
print(len(inputs))

100


In [7]:
outputs = llm.generate(inputs, sampling_params)
response = [output.outputs[0].text for output in outputs]

Processed prompts: 100%|██████████| 100/100 [00:03<00:00, 26.28it/s, est. speed input: 578.12 toks/s, output: 2728.21 toks/s]


In [8]:
rougeLs = rouge.compute(predictions=response, references=[base_data["response"]]*100)["rougeL"]
print(rougeLs)

0.24712938561472625


In [ ]:
# rougeLs = []
# for input in tqdm(base_data):
#     response = generate_one_completion(template.format_map({"instruction":input["instruction"]}))
#     rougeLs.append(rouge.compute(predictions=[response.split("Response:")[1]], references=[input["response"]])["rougeL"]) #unsloth
#     rougeLs.append(rouge.compute(predictions=[response], references=[input["response"]])["rougeL"])
# print(sum(rougeLs)/len(base_data))